In [2]:
#library import
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import *
spark=SparkSession.builder.master("local").appName("unified_to_refinemant").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/07 15:58:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/02/07 15:58:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/07 15:58:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/02/07 15:58:33 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/02/07 15:58:33 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/02/07 15:58:33 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [16]:
#script execution for raw data to landing
class unified_to_refinement:
    
    """Script to read parquet file from local path to unified.."""
    
    def __init__(self,file_parameter):
        
        """function to get input parameters to read the file from local path"""
        self.file_parameter=file_parameter
        
        """read parquet data from local"""
    def read_parquet(self,fileformat,file_path):
        spark.read.parquet(file_path).show()
        return spark.read.parquet(file_path)

        """save file as parquet and also save as table in local db"""
    def write_parquet(self,df,dest_path,table_name):   
        print(dest_path,table_name)
        df.write.mode("overwrite").parquet(dest_path)
        df.coalesce(1).write.format(self.file_parameter['parquet']).mode("overwrite").option("path",dest_path)\
        .saveAsTable("school_refinement.{0}".format(table_name))
        
        
        """creating temporary view to query the 1st rank students from school unified_data"""
        
    def create_view(self,df):
        df.createOrReplaceTempView(self.file_parameter['temp_table'])
        temp_df = spark.sql("select * from {0}".format(self.file_parameter['temp_table']))
        temp_df.show(truncate = False)
        highest_marks_df = spark.sql("select * from (select *, dense_rank() over(partition by subject_name, month(exam_date) order by marks DESC)as highest_mark  from Refinement_student_details)A where A.highest_mark = 1 order by month(exam_date)")
        return highest_marks_df
    
    def main_block(self):
        
        """main execution block"""
        spark.sql("create database if not exists {0}".format(self.file_parameter['local_db_name']))
        students_details=self.read_parquet(self.file_parameter['parquet_format'],self.file_parameter['src_base_read_path']+self.file_parameter['read_filename_school_unified']+self.file_parameter['parquet_format']);
       
        parquet_df1 = self.create_view(students_details);
        self.write_parquet(parquet_df1,self.file_parameter['dest_base_write_path']+self.file_parameter['write_merged_df'],self.file_parameter['table_name_school_refinement']);

        
        

In [17]:
if __name__=='__main__':
    
    file_read={
        "csv_format":".csv",
        "csv":"csv",
        "parquet_format":".parquet",
        "parquet":"parquet",
        "table_name_school_refinement":"students_highestmarks",
        "table_name_students":"students",
        "table_name_subject":"subjects",
        "read_filename_marks":"/marks/markdetails",
        "read_filename_student":"/student/studentdetails",
        "read_filename_subject":"/subject/subjectdetails",
        "read_filename_school_unified":"/school_unified/students_details",
        
        "write_merged_df":"/school_Refinements/students_highestmarks.parquet",
        "dest_base_write_path":"../../../../git projects/big_data_project_git/data_files/refinement",
        "src_base_read_path":"../../../../git projects/big_data_project_git/data_files/unified",
        "temp_table":"Refinement_student_details",
        "local_db_name":"school_refinement",

    }
   
    

    refinement_obj1=unified_to_refinement(file_read)
    refinement_obj1.main_block()
   

+----------+----------+----------+-----+----------------------------+-----------------------------+----------------------------+-----------------------------+------------+------------------------------+-------------------------------+------------------------------+-------------------------------+------------+------------------------------+-------------------------------+------------------------------+-------------------------------+
|subject_id|student_id| exam_date|marks|marks_audit_created_username|marks_audit_created_timestamp|marks_audit_updated_username|marks_audit_updated_timestamp|student_name|student_audit_created_username|student_audit_created_timestamp|student_audit_updated_username|student_audit_updated_timestamp|subject_name|subject_audit_created_username|subject_audit_created_timestamp|subject_audit_updated_username|subject_audit_updated_timestamp|
+----------+----------+----------+-----+----------------------------+-----------------------------+---------------------------

+----------+----------+----------+-----+----------------------------+-----------------------------+----------------------------+-----------------------------+------------+------------------------------+-------------------------------+------------------------------+-------------------------------+------------+------------------------------+-------------------------------+------------------------------+-------------------------------+
|subject_id|student_id|exam_date |marks|marks_audit_created_username|marks_audit_created_timestamp|marks_audit_updated_username|marks_audit_updated_timestamp|student_name|student_audit_created_username|student_audit_created_timestamp|student_audit_updated_username|student_audit_updated_timestamp|subject_name|subject_audit_created_username|subject_audit_created_timestamp|subject_audit_updated_username|subject_audit_updated_timestamp|
+----------+----------+----------+-----+----------------------------+-----------------------------+---------------------------

22/02/07 22:35:36 WARN HadoopFSUtils: The directory file:/Users/vasanth_ku/Vasanth/spark-apache/git%20projects/big_data_project_git/git projects/big_data_project_git/data_files/refinement/school_Refinements/students_highestmarks.parquet was not found. Was it deleted very recently?
